In [63]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
url = 'http://www.yellowpages.com/search?search_terms=coffee&geo_location_terms=Los+Angeles%2C+CA'
r = requests.get(url)

In [10]:
# pass the request's content into BeautifulSoup
# the second argument is the HTML parser (don't worry about it)
soup = BeautifulSoup( r.content, 'lxml' )

In [20]:
# e.g. extracting all the URLs found within a page’s <a> tags:
# find the <a> tags and get the href part
links = soup.find_all('a')[:10]

# obtain the text in the tags and the 'href' attribute
# .text is the same as .get_text()
# .get('href') is the same as .attrs['href']
for link in links:
    print( link.text, link.get('href') )

 /
Find People /whitepages
Advertise With Us http://adsolutions.yp.com
Find People /whitepages
Featured Collections /collections
Restaurants /restaurants?ic=home_restaurants_landing_page
Automotive /los-angeles-ca/automotive
Home Improvement /los-angeles-ca/home-improvement
Health & Wellness /los-angeles-ca/health-and-wellness
Attorneys /los-angeles-ca/attorneys


In [26]:
# obtaining the div tag that has the class of info
data = soup.find_all( 'div', { 'class': 'info' } )

In [50]:
# use one of the div and print out its child .contents
for d in data[:1]:
    
    # access to the first child
    content = d.contents[0]
    print(content) 
    print()
    
    # but calling .text gives you a number the restaurant name and menu
    # to only get the restaurant name we can do another find_all
    print(content.text)
    print()
    
    # access the business name class, there's only 1 element, so access it with 0,
    # note that we don't have to access to the child to get this
    print(content.find_all( 'a', { 'class', 'business-name' } )[0].text )
    print()
    
    # we can also obtain the restaurant's cellphone
    # this time its in the second child
    phone = d.contents[1].find_all( 'div', { 'class': 'phone' } )
    print(phone)
    # notice that the class is 'phones phone primary', this is 
    # three separate class and you can use any one of it to access it
    
    phone = d.contents[1].find_all( 'div', { 'class': 'phones' } )[0]
    print(phone.text)

<h3 class="n">1. <a class="business-name" data-analytics='{"target":"name","feature_click":""}' href="/los-angeles-ca/mip/caffe-latte-1050840?lid=1050840" itemprop="name" rel="">Caffe Latte</a><a class="menu" data-analytics='{"click_id":1614,"target":"menus","listing_features":"menu-link"}' href="/listings/1050840/menu?lid=1050840">Menu</a></h3>

1. Caffe LatteMenu

Caffe Latte

[<div class="phones phone primary" itemprop="telephone">(323) 936-5213</div>]
(323) 936-5213


In [58]:
# loop through all data to get the restaurant name
phones = []
businesses = []

for d in data:
    business_name = d.find_all( 'a', { 'class', 'business-name' } )[0].text
    
    # since not all business have a phone number, you'll get an list index
    # out of range if you access an empty list with [0]
    phone_info = d.find_all( 'div', { 'class': 'phones' } )
    if phone_info:
        phone_number = phone_info[0].text
    else:
        phone_number = None
    
    businesses.append(business_name)
    phones.append(phone_number)

In [64]:
# we can now store it as a DataFrame
business_df = pd.DataFrame({
    'businesses': businesses,
    'phones': phones
})
business_df.head()

,businesses,phones
0,Caffe Latte,(323) 936-5213
1,Bourgeois Pig,(323) 464-6008
2,Beard Papa Sweet Cafe,(323) 462-6100
3,Intelligentsia Coffee,(323) 663-6173
4,Groundwork Coffee Inc,(323) 871-0143




- [Scrape Websites with Python + Beautiful Soup 4 + Requests](https://www.youtube.com/watch?v=3xQTJi2tqgk&index=1&list=PLEsfXFp6DpzR6FatOy4RtoXfu4PeYO_RL)